In [1]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from scipy import stats
import math
from sklearn import linear_model
from sklearn import model_selection
from numpy import mean
from numpy import std
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import accuracy_score

In [3]:
PLAYERS_NUMBER = 6
ages = [26, 15, 14, 13, 19, 17]
levels = [1, 2, 1, 2, 1, 2]
backhands = [2, 1, 2, 2, 2, 1]
smash_types = ['Dretes', 'Reves', 'Serve', 'Smash', 'VD', 'VR']
sensor_types = ['ACC', 'GYR']
stroke_numbers = {'Dretes': 30, 'Reves': 30, 'Serve': 15, 'Smash': 15, 'VD': 20, 'VR': 20}

In [24]:
"""FUNCTION TO LOAD ALL SENSOR DATA IN A COMBINED DATASET"""

def load_data (player_number, ages, levels, backhands, smash_types, sensor_types):
    result = []
    interval = player_number * len(smash_types)
    for i in range(interval * len(sensor_types)):
        player = int(i /(len(smash_types) * len(sensor_types)))
        smash = i - player_number * int(i / player_number)
        sensor =  int(i / player_number) % 2
        df = pd.read_csv(f'./Jugador {player+1}/Enregistraments J{player+1}/J{player+1} {smash_types[smash]} {sensor_types[sensor]}.csv').sort_values('UNIX_TIMESTAMP')
        df.drop(columns=['DATE'], inplace=True)
        df.rename(columns={ df.columns[0]: 'X', df.columns[1]: 'Y', df.columns[2]: 'Z', df.columns[3]: 'TIMESTAMP'}, inplace = True)
        df['AGE'] = ages[player]
        df['LEVEL'] = levels[player]
        df['BACKHANDS'] = backhands[player]
        df['SMASH'] = smash
        df['SENSOR'] = sensor
        temp = np.array_split(df, stroke_numbers[smash_types[smash]])
        for j in range(stroke_numbers[smash_types[smash]]):
            dfTemp = temp[j] 
            dfTemp['GROUP'] = str(i+1) + str(j+1)
        result += temp
    return result

data = load_data(PLAYERS_NUMBER, ages, levels, backhands, smash_types, sensor_types)
pd.concat(data)

,X,Y,Z,TIMESTAMP,AGE,LEVEL,BACKHANDS,SMASH,SENSOR,GROUP
0,0.013325,0.008126,0.000599,1697802318975,26,1,2,0,0,11
1,-0.110110,-0.069654,-0.005859,1697802318996,26,1,2,0,0,11
2,-0.068091,-0.044559,-0.003320,1697802319016,26,1,2,0,0,11
3,0.019304,0.012501,0.000792,1697802319038,26,1,2,0,0,11
4,0.186550,0.116849,0.005761,1697802319056,26,1,2,0,0,11
...,...,...,...,...,...,...,...,...,...,...
7358,0.040840,0.128904,-0.097510,1697903854058,17,2,1,5,1,7220
7359,0.034853,0.110147,-0.083276,1697903854068,17,2,1,5,1,7220
7360,0.034720,0.108551,-0.085271,1697903854079,17,2,1,5,1,7220
7361,0.034587,0.107221,-0.086867,1697903854090,17,2,1,5,1,7220


In [38]:
def calculate_statistics(df, suffix):
    row = {}
    row['x_mean_' + suffix] = [df['X'].mean()]
    row['y_mean_' + suffix] = [df['Y'].mean()]
    row['z_mean_' + suffix] = [df['Z'].mean()]
    row['x_std_' + suffix] = [df['X'].std()]
    row['y_std_' + suffix] = [df['Y'].std()]
    row['z_std_' + suffix] = [df['Z'].std()]
    row['x_min_' + suffix] = [df['X'].min()]
    row['y_min_' + suffix] = [df['Y'].min()]
    row['z_min_' + suffix] = [df['Z'].min()]
    row['x_max_' + suffix] = [df['X'].max()]
    row['y_max_' + suffix] = [df['Y'].max()]
    row['z_max_' + suffix] = [df['Z'].max()]
    row['x_max_' + suffix] = [df['X'].max()]
    row['y_max_' + suffix] = [df['Y'].max()]
    row['z_max_' + suffix] = [df['Z'].max()]
    row['x_median_' + suffix] = [df['X'].median()]
    row['y_median_' + suffix] = [df['Y'].median()]
    row['z_median_' + suffix] = [df['Z'].median()]
    row['x_mad_' + suffix] = [np.median(np.absolute(df['X'] - row['x_median_' + suffix]))] # median absolute difference
    row['y_mad_' + suffix] = [np.median(np.absolute(df['Y'] - row['y_median_' + suffix]))]
    row['z_mad_' + suffix] = [np.median(np.absolute(df['Z'] - row['z_median_' + suffix]))]
    row['x_iqr_' + suffix] = [np.percentile(df['X'], 75) - np.percentile(df['X'], 25)]
    row['y_iqr_' + suffix] = [np.percentile(df['Y'], 75) - np.percentile(df['Y'], 25)]
    row['z_iqr_' + suffix] = [np.percentile(df['Z'], 75) - np.percentile(df['Z'], 25)]
    row['x_pcount_' + suffix] = [np.sum(df['X'] > 0)]
    row['y_pcount_' + suffix] = [np.sum(df['Y'] > 0)]
    row['z_pcount_' + suffix] = [np.sum(df['Z'] > 0)]
    row['x_ncount_' + suffix] = [np.sum(df['X'] < 0)]
    row['y_ncount_' + suffix] = [np.sum(df['Y'] < 0)]
    row['z_ncount_' + suffix] = [np.sum(df['Z'] < 0)]
    row['x_abvmean_' + suffix] = [np.sum(df['X'] > df['X'].mean())]
    row['y_abvmean_' + suffix] = [np.sum(df['Y'] > df['Y'].mean())]
    row['z_abvmean_' + suffix] = [np.sum(df['Z'] > df['Z'].mean())]
    row['x_cntpeaks_' + suffix] = [len(find_peaks(df['X'])[0])]
    row['y_cntpeaks_' + suffix] = [len(find_peaks(df['Y'])[0])]
    row['z_cntpeaks_' + suffix] = [len(find_peaks(df['Z'])[0])]
    row['x_skew_' + suffix] = [stats.skew(df['X'])]
    row['y_skew_' + suffix] = [stats.skew(df['Y'])]
    row['z_skew_' + suffix] = [stats.skew(df['Z'])]
    row['x_kurt_' + suffix] = [stats.kurtosis(df['X'])]
    row['y_kurt_' + suffix] = [stats.kurtosis(df['Y'])]
    row['z_kurt_' + suffix] = [stats.kurtosis(df['Z'])]
    row['time'] = df.iloc[-1, df.columns.get_loc('UNIX_TIMESTAMP')] - df.iloc[0, df.columns.get_loc('UNIX_TIMESTAMP')]
    if (suffix == 'acc' or suffix == 'ftt_acc'):
        row['x_energy_' + suffix] = [np.sum(df['X']**2)/100]
        row['y_energy_' + suffix] = [np.sum(df['Y']**2)/100]
        row['z_energy_' + suffix] = [np.sum(df['Z']**2)/100]
        row['acceleration'] = [np.mean((df['X']**2 + df['Y']**2 + df['Z']**2)**0.5)]
    return pd.DataFrame.from_dict(row)

def load_csv(smash_types, stroke_numbers, player, smash, sensor):
    df_acc = pd.read_csv(f'./Jugador {player+1}/Enregistraments J{player+1}/J{player+1} {smash_types[smash]} {sensor}.csv').sort_values('UNIX_TIMESTAMP')
    df_acc.rename(columns={ df_acc.columns[0]: 'X', df_acc.columns[1]: 'Y', df_acc.columns[2]: 'Z'}, inplace = True)
    temp = np.array_split(df_acc, stroke_numbers[smash_types[smash]])
    return temp

In [39]:
"""FUNCTION TO LOAD STATISTICAL DATA FOR EACH STROKE"""

def load_data (player_number, ages, levels, backhands, smash_types, stroke_numbers, sensor_types):
    interval = player_number * len(smash_types)
    result = []
    for i in range(interval):
        player = int(i /len(smash_types))
        smash = i - player_number * int(i / player_number)
        temp = load_csv(smash_types, stroke_numbers, player, smash, sensor_types[0])
        temp2 = load_csv(smash_types, stroke_numbers, player, smash, sensor_types[1])
        for j in range(len(temp)):
            df = temp[j]
            row1 = calculate_statistics(df, 'acc')
            df2 = temp2[j]
            row3 = calculate_statistics(df2, 'gyr')

            row_result = pd.concat([row1, row3], axis=1)
            row_result['player'] = float(player)
            row_result['age'] = float(ages[player])
            row_result['level'] = float(levels[player])
            row_result['backhands'] = float(backhands[player])
            row_result['smash'] = float(smash)

            df['distance_xy'] = 0
            df['distance_xz'] = 0
            df['distance_yz'] = 0
            df['angle_z'] = 0
            df['angle_y'] = 0
            df['angle_x'] = 0
            df['xy_x'] = 0
            df['xy_y'] = 0
            df['xz_x'] = 0
            df['xz_y'] = 0
            df['yz_x'] = 0
            df['yz_y'] = 0

            for k in range(len(df)): 
                if k != 0:
                    time = df.iloc[k, df.columns.get_loc('UNIX_TIMESTAMP')] - df.iloc[k-1, df.columns.get_loc('UNIX_TIMESTAMP')]
                else: 
                    time = 0
                df.iloc[k, df.columns.get_loc('distance_xy')] = math.sqrt(df.iloc[k, df.columns.get_loc('X')]**2*(time)**4 + df.iloc[k, df.columns.get_loc('Y')]**2*time**4)
                df.iloc[k, df.columns.get_loc('distance_xz')] = math.sqrt(df.iloc[k, df.columns.get_loc('X')]**2*(time)**4 + df.iloc[k, df.columns.get_loc('Z')]**2*time**4)
                df.iloc[k, df.columns.get_loc('distance_yz')] = math.sqrt(df.iloc[k, df.columns.get_loc('Y')]**2*(time)**4 + df.iloc[k, df.columns.get_loc('Z')]**2*time**4)
                if k != 0:
                    z = k
                else:
                    z = -1
                df.iloc[k, df.columns.get_loc('angle_z')] = (df2.iloc[z+1, df.columns.get_loc('Z')] + df2.iloc[z+2, df.columns.get_loc('Z')])*time
                df.iloc[k, df.columns.get_loc('angle_y')] = (df2.iloc[z+1, df.columns.get_loc('Y')] + df2.iloc[z+2, df.columns.get_loc('Y')])*time
                df.iloc[k, df.columns.get_loc('angle_x')] = (df2.iloc[z+1, df.columns.get_loc('X')] + df2.iloc[z+2, df.columns.get_loc('X')])*time

                df.iloc[k, df.columns.get_loc('xy_x')] = df.iloc[k, df.columns.get_loc('distance_xy')] * math.cos(df.iloc[k, df.columns.get_loc('angle_z')])
                df.iloc[k, df.columns.get_loc('xy_y')] = df.iloc[k, df.columns.get_loc('distance_xy')] * math.sin(df.iloc[k, df.columns.get_loc('angle_z')])
                df.iloc[k, df.columns.get_loc('xz_x')] = df.iloc[k, df.columns.get_loc('distance_xz')] * math.cos(df.iloc[k, df.columns.get_loc('angle_y')])
                df.iloc[k, df.columns.get_loc('xz_y')] = df.iloc[k, df.columns.get_loc('distance_xz')] * math.sin(df.iloc[k, df.columns.get_loc('angle_y')])
                df.iloc[k, df.columns.get_loc('yz_x')] = df.iloc[i, df.columns.get_loc('distance_yz')] * math.cos(df.iloc[k, df.columns.get_loc('angle_x')])
                df.iloc[k, df.columns.get_loc('yz_y')] = df.iloc[i, df.columns.get_loc('distance_yz')] * math.sin(df.iloc[k, df.columns.get_loc('angle_x')])

            # AÑADO LA DISTANCIA TOTAL, EL ÁNGULO TOTAL, EL PUNTO FINAL Y EL PUNTO INTERMEDIO CALCULADOS (SE PUEDEN AÑADIR MÁS PUNTOS PARA TENER MÁS DATOS)
            row_result['distance_xy'] = df['distance_xy'].sum()
            row_result['distance_xz'] = df['distance_xz'].sum()
            row_result['distance_yz'] = df['distance_yz'].sum()
            row_result['angle_z'] = df['angle_x'].sum()
            row_result['angle_y'] = df['angle_y'].sum()
            row_result['angle_x'] = df['angle_z'].sum()
            row_result['xy_y_last'] = df.iloc[stroke_numbers[smash_types[smash]] - 1, df.columns.get_loc('xy_y')]
            row_result['xy_y_middle']= df.iloc[stroke_numbers[smash_types[smash]]//2, df.columns.get_loc('xy_y')]
            row_result['xy_x_last'] = df.iloc[stroke_numbers[smash_types[smash]] - 1, df.columns.get_loc('xy_x')]
            row_result['xy_x_middle']= df.iloc[stroke_numbers[smash_types[smash]]//2, df.columns.get_loc('xy_x')]
            row_result['xz_y_last'] = df.iloc[stroke_numbers[smash_types[smash]] - 1, df.columns.get_loc('xz_y')]
            row_result['xz_y_middle']= df.iloc[stroke_numbers[smash_types[smash]]//2, df.columns.get_loc('xz_y')]
            row_result['xz_x_last'] = df.iloc[stroke_numbers[smash_types[smash]] - 1, df.columns.get_loc('xz_x')]
            row_result['xz_x_middle']= df.iloc[stroke_numbers[smash_types[smash]]//2, df.columns.get_loc('xz_x')]
            row_result['yz_y_last'] = df.iloc[stroke_numbers[smash_types[smash]] - 1, df.columns.get_loc('xz_y')]
            row_result['yz_y_middle']= df.iloc[stroke_numbers[smash_types[smash]]//2, df.columns.get_loc('yz_y')]
            row_result['yz_x_last'] = df.iloc[stroke_numbers[smash_types[smash]] - 1, df.columns.get_loc('yz_x')]
            row_result['yz_x_middle']= df.iloc[stroke_numbers[smash_types[smash]]//2, df.columns.get_loc('yz_x')]
            
            result.append(row_result)
    return pd.concat(result)

data = load_data(PLAYERS_NUMBER, ages, levels, backhands, smash_types, stroke_numbers, sensor_types)
data

,x_mean_acc,y_mean_acc,z_mean_acc,x_std_acc,y_std_acc,z_std_acc,x_min_acc,y_min_acc,z_min_acc,x_max_acc,y_max_acc,z_max_acc,x_median_acc,y_median_acc,z_median_acc,x_mad_acc,y_mad_acc,z_mad_acc,x_iqr_acc,y_iqr_acc,z_iqr_acc,x_pcount_acc,y_pcount_acc,z_pcount_acc,x_ncount_acc,y_ncount_acc,z_ncount_acc,x_abvmean_acc,y_abvmean_acc,z_abvmean_acc,x_cntpeaks_acc,y_cntpeaks_acc,z_cntpeaks_acc,x_skew_acc,y_skew_acc,z_skew_acc,x_kurt_acc,y_kurt_acc,z_kurt_acc,time,x_energy_acc,y_energy_acc,z_energy_acc,acceleration,x_mean_gyr,y_mean_gyr,z_mean_gyr,x_std_gyr,y_std_gyr,z_std_gyr,x_min_gyr,y_min_gyr,z_min_gyr,x_max_gyr,y_max_gyr,z_max_gyr,x_median_gyr,y_median_gyr,z_median_gyr,x_mad_gyr,y_mad_gyr,z_mad_gyr,x_iqr_gyr,y_iqr_gyr,z_iqr_gyr,x_pcount_gyr,y_pcount_gyr,z_pcount_gyr,x_ncount_gyr,y_ncount_gyr,z_ncount_gyr,x_abvmean_gyr,y_abvmean_gyr,z_abvmean_gyr,x_cntpeaks_gyr,y_cntpeaks_gyr,z_cntpeaks_gyr,x_skew_gyr,y_skew_gyr,z_skew_gyr,x_kurt_gyr,y_kurt_gyr,z_kurt_gyr,time,player,age,level,backhands,smash,distance_xy,distance_xz,distance_yz,angle_z,angle_y,angle_x,xy_y_last,xy_y_middle,xy_x_last,xy_x_middle,xz_y_last,xz_y_middle,xz_x_last,xz_x_middle,yz_y_last,yz_y_middle,yz_x_last,yz_x_middle
0,0.119649,0.075253,-0.011181,0.056442,0.036162,0.011034,-0.110110,-0.069654,-0.029241,0.333318,0.198022,0.007792,0.129523,0.079631,-0.011266,0.037786,0.025356,0.010673,0.072021,0.050243,0.021003,127,127,27,3,3,103,75,74,65,9,9,5,-0.550520,-0.607466,-0.099980,2.905261,2.179081,-1.537183,2614,0.022720,0.009049,0.000320,0.145690,-0.346136,-0.289389,0.029494,0.287170,0.198342,0.087777,-0.980284,-0.889825,-0.254882,0.004922,0.004922,0.386447,-0.316807,-0.314412,0.013236,0.235393,0.145666,0.019821,0.482427,0.259737,0.052546,25,28,186,235,230,73,144,120,104,30,25,29,-0.539721,-0.105965,0.206929,-0.768824,-0.565476,3.489603,2634,0.0,26.0,1.0,2.0,0.0,7751.362958,6600.962656,4191.654719,-730.794677,-908.621519,88.569213,-4.485667,2.516997,35.684831,60.030828,-0.817015,9.925753,30.701116,50.109120,-0.817015,0.0,0.0,0.0
0,0.158366,0.084453,-0.044530,0.064365,0.034816,0.025279,0.000542,0.000191,-0.101574,0.280684,0.144491,-0.000102,0.152324,0.087056,-0.043000,0.050265,0.026830,0.020106,0.106840,0.052667,0.036034,130,130,0,0,0,130,63,66,68,4,3,3,0.031116,-0.248425,-0.484940,-0.808359,-0.773329,-0.605872,2614,0.037948,0.010836,0.003402,0.185495,-0.393011,-0.227487,0.215061,0.426748,0.547005,0.196675,-1.263235,-1.111982,-0.076757,0.256611,0.654765,0.640265,-0.307095,-0.065250,0.179255,0.254416,0.442917,0.146131,0.580103,1.029372,0.322227,44,121,226,216,139,34,169,145,104,6,6,8,-0.610069,-0.188370,0.512483,-0.651549,-1.404568,-0.709934,2625,0.0,26.0,1.0,2.0,0.0,9583.617828,8808.333980,5134.556346,-3574.799940,-1986.363467,1537.315632,39.244966,-77.443169,96.189802,-22.048062,-85.554220,-8.288122,45.291439,70.313240,-85.554220,0.0,0.0,-0.0
0,0.705115,0.575456,-0.977364,0.800909,1.265109,1.529748,-0.274389,-0.661758,-5.194325,2.415067,3.896989,0.082945,0.494091,0.015037,-0.249011,0.646356,0.117098,0.301923,1.460489,0.668150,1.095494,94,68,36,36,62,94,57,34,94,2,3,2,0.479716,1.587956,-1.692850,-1.046606,1.130063,1.539312,2614,1.473821,2.495141,4.260577,1.598933,-1.329833,-0.629818,1.348140,1.904659,3.033405,3.271360,-7.805427,-19.245981,-0.814797,1.354094,4.870294,21.817148,-0.870669,-0.303703,0.231868,1.006691,0.795442,0.751809,1.936058,1.487820,2.358656,68,104,151,192,156,109,160,159,81,6,10,9,-1.550616,-3.350901,4.115060,2.395200,16.671190,19.570853,2623,0.0,26.0,1.0,2.0,0.0,64039.823768,73550.805654,67811.514720,-3408.123687,-8271.828911,6724.809839,-38.913111,-149.488290,33.426953,23.089789,19.520663,-81.484065,-48.340884,121.528266,19.520663,-0.0,-0.0,0.0
0,0.750097,1.026938,-1.270542,0.998745,1.910227,1.902650,-0.722997,-0.250944,-6.398224,3.125900,5.935603,0.295709,0.727732,0.067038,-0.447860,0.778379,0.174045,0.583183,1.519565,1.099968,1.671153,92,76,38,38,54,92,65,33,89,3,3,2,0.594898,1.602820,-1.562669,-0.296258,1.004371,1.222706,2614,2.018204,6.078147,6.768461,2.

In [41]:
data = data.reset_index()
data = data.drop('index', axis=1)
pd.set_option('display.max_columns', None)
data.to_csv('final_data')
data

,x_mean_acc,y_mean_acc,z_mean_acc,x_std_acc,y_std_acc,z_std_acc,x_min_acc,y_min_acc,z_min_acc,x_max_acc,y_max_acc,z_max_acc,x_median_acc,y_median_acc,z_median_acc,x_mad_acc,y_mad_acc,z_mad_acc,x_iqr_acc,y_iqr_acc,z_iqr_acc,x_pcount_acc,y_pcount_acc,z_pcount_acc,x_ncount_acc,y_ncount_acc,z_ncount_acc,x_abvmean_acc,y_abvmean_acc,z_abvmean_acc,x_cntpeaks_acc,y_cntpeaks_acc,z_cntpeaks_acc,x_skew_acc,y_skew_acc,z_skew_acc,x_kurt_acc,y_kurt_acc,z_kurt_acc,time,x_energy_acc,y_energy_acc,z_energy_acc,acceleration,x_mean_gyr,y_mean_gyr,z_mean_gyr,x_std_gyr,y_std_gyr,z_std_gyr,x_min_gyr,y_min_gyr,z_min_gyr,x_max_gyr,y_max_gyr,z_max_gyr,x_median_gyr,y_median_gyr,z_median_gyr,x_mad_gyr,y_mad_gyr,z_mad_gyr,x_iqr_gyr,y_iqr_gyr,z_iqr_gyr,x_pcount_gyr,y_pcount_gyr,z_pcount_gyr,x_ncount_gyr,y_ncount_gyr,z_ncount_gyr,x_abvmean_gyr,y_abvmean_gyr,z_abvmean_gyr,x_cntpeaks_gyr,y_cntpeaks_gyr,z_cntpeaks_gyr,x_skew_gyr,y_skew_gyr,z_skew_gyr,x_kurt_gyr,y_kurt_gyr,z_kurt_gyr,time,player,age,level,backhands,smash,distance_xy,distance_xz,distance_yz,angle_z,angle_y,angle_x,xy_y_last,xy_y_middle,xy_x_last,xy_x_middle,xz_y_last,xz_y_middle,xz_x_last,xz_x_middle,yz_y_last,yz_y_middle,yz_x_last,yz_x_middle
0,0.119649,0.075253,-0.011181,0.056442,0.036162,0.011034,-0.110110,-0.069654,-0.029241,0.333318,0.198022,0.007792,0.129523,0.079631,-0.011266,0.037786,0.025356,0.010673,0.072021,0.050243,0.021003,127,127,27,3,3,103,75,74,65,9,9,5,-0.550520,-0.607466,-0.099980,2.905261,2.179081,-1.537183,2614,0.022720,0.009049,0.000320,0.145690,-0.346136,-0.289389,0.029494,0.287170,0.198342,0.087777,-0.980284,-0.889825,-0.254882,0.004922,0.004922,0.386447,-0.316807,-0.314412,0.013236,0.235393,0.145666,0.019821,0.482427,0.259737,0.052546,25,28,186,235,230,73,144,120,104,30,25,29,-0.539721,-0.105965,0.206929,-0.768824,-0.565476,3.489603,2634,0.0,26.0,1.0,2.0,0.0,7751.362958,6600.962656,4191.654719,-730.794677,-908.621519,88.569213,-4.485667,2.516997,35.684831,60.030828,-0.817015,9.925753,30.701116,50.109120,-0.817015,0.0,0.0,0.0
1,0.158366,0.084453,-0.044530,0.064365,0.034816,0.025279,0.000542,0.000191,-0.101574,0.280684,0.144491,-0.000102,0.152324,0.087056,-0.043000,0.050265,0.026830,0.020106,0.106840,0.052667,0.036034,130,130,0,0,0,130,63,66,68,4,3,3,0.031116,-0.248425,-0.484940,-0.808359,-0.773329,-0.605872,2614,0.037948,0.010836,0.003402,0.185495,-0.393011,-0.227487,0.215061,0.426748,0.547005,0.196675,-1.263235,-1.111982,-0.076757,0.256611,0.654765,0.640265,-0.307095,-0.065250,0.179255,0.254416,0.442917,0.146131,0.580103,1.029372,0.322227,44,121,226,216,139,34,169,145,104,6,6,8,-0.610069,-0.188370,0.512483,-0.651549,-1.404568,-0.709934,2625,0.0,26.0,1.0,2.0,0.0,9583.617828,8808.333980,5134.556346,-3574.799940,-1986.363467,1537.315632,39.244966,-77.443169,96.189802,-22.048062,-85.554220,-8.288122,45.291439,70.313240,-85.554220,0.0,0.0,-0.0
2,0.705115,0.575456,-0.977364,0.800909,1.265109,1.529748,-0.274389,-0.661758,-5.194325,2.415067,3.896989,0.082945,0.494091,0.015037,-0.249011,0.646356,0.117098,0.301923,1.460489,0.668150,1.095494,94,68,36,36,62,94,57,34,94,2,3,2,0.479716,1.587956,-1.692850,-1.046606,1.130063,1.539312,2614,1.473821,2.495141,4.260577,1.598933,-1.329833,-0.629818,1.348140,1.904659,3.033405,3.271360,-7.805427,-19.245981,-0.814797,1.354094,4.870294,21.817148,-0.870669,-0.303703,0.231868,1.006691,0.795442,0.751809,1.936058,1.487820,2.358656,68,104,151,192,156,109,160,159,81,6,10,9,-1.550616,-3.350901,4.115060,2.395200,16.671190,19.570853,2623,0.0,26.0,1.0,2.0,0.0,64039.823768,73550.805654,67811.514720,-3408.123687,-8271.828911,6724.809839,-38.913111,-149.488290,33.426953,23.089789,19.520663,-81.484065,-48.340884,121.528266,19.520663,-0.0,-0.0,0.0
3,0.750097,1.026938,-1.270542,0.998745,1.910227,1.902650,-0.722997,-0.250944,-6.398224,3.125900,5.935603,0.295709,0.727732,0.067038,-0.447860,0.778379,0.174045,0.583183,1.519565,1.099968,1.671153,92,76,38,38,54,92,65,33,89,3,3,2,0.594898,1.602820,-1.562669,-0.296258,1.004371,1.222706,2614,2.018204,6.078147,6.768461,2.

## REGRESSION, SVC, KNEIGHBORS

In [6]:
# ACTIVAR PARA CARGAR LOS DATOS DIRECTAMENTE
# data = pd.read_csv('final_data')

X = np.array(data.drop(['smash'], axis=1))
y = np.array(data['smash'])


In [55]:
seed = 2
X = np.array(data.drop(['smash'], axis=1))
y = np.array(data['smash'])
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=seed)
model = svm.SVC()
model.fit(X_train, Y_train)
model.score(X_test, Y_test)


0.30128205128205127

In [58]:
seed = 2
X = np.array(data.drop(['smash'], axis=1))
y = np.array(data['smash'])
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=seed)
model = KNeighborsClassifier(n_neighbors=6)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6153846153846154

In [43]:
X = np.array(data.drop(['smash'], axis=1))
y = np.array(data['smash'])

seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=seed)
model = linear_model.LogisticRegression(max_iter=1000)
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=3, random_state=7)

cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f %s (%f)" % (name, cv_results.mean(), "+-", cv_results.std())
model.fit(X_train, Y_train)
predictions = model.predict(X_test)
print(model.score(X=X_test, y=Y_test))
print(accuracy_score(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: 

0.5576923076923077
0.5576923076923077
[[31  1  1  2  2  1]
 [ 1 18  2  1  3  1]
 [ 0  3 16  2  0  0]
 [ 3  3  3  2  1  1]
 [ 5 10  0  4  8  6]
 [ 3  7  1  1  1 12]]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## TREES

In [59]:
k_fold = RepeatedKFold(n_splits=20, n_repeats=3, random_state=1)
val_score = []
train_score = []
n_stimators = 1500
model = RandomForestClassifier(warm_start=True)

for i, (train, val) in enumerate(k_fold.split(X_train)):
    print("Iteración:", i+1)
    print("val_size:", len(val))

    model.fit(X_train[train], Y_train[train])
    model.n_estimators += 100

    score_val = model.score(X_train[val], Y_train[val])
    val_score.append(score_val)
    score_train = model.score(X_train[train], Y_train[train])
    train_score.append(score_train)
    print("score_train:", score_train)
    print("score_val:", score_val)

    if i % 1000 == 0: 
        pickle.dump(model, open("model_saved.sav", "wb"))
    if score_val < np.mean(val_score) // 2:
        print("Se ha encontrado una bajada importante de accuracy en validación")
        print("Este es el responsable:")
        print(X_train[val])
        break 
    if np.mean(val_score) >= 0.99 and len(val_score) > 20:
        pickle.dump(model, open("model_saved.sav", "wb"))
        print("STOP")
        break
    print("##############")

Iteración: 1
val_size: 32
score_train: 1.0
score_val: 0.875
##############
Iteración: 2
val_size: 32
score_train: 1.0
score_val: 1.0
##############
Iteración: 3
val_size: 32
score_train: 1.0
score_val: 1.0
##############
Iteración: 4
val_size: 32
score_train: 1.0
score_val: 1.0
##############
Iteración: 5
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 6
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 7
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 8
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 9
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 10
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 11
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 12
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 13
val_size: 31
score_train: 1.0
score_val: 1.0
##############
Iteración: 14
val_size: 31
score_train: 1.0
s

In [60]:
predictions = model.predict(X_test)
print(model.score(X=X_test, y=Y_test))
print(accuracy_score(Y_test, predictions))
print(confusion_matrix(Y_test, predictions))

0.9038461538461539
0.9038461538461539
[[29  2  0  2  1  0]
 [ 1 37  0  0  0  0]
 [ 2  0 25  0  0  0]
 [ 1  2  1 18  0  0]
 [ 0  0  0  0 14  2]
 [ 0  0  0  0  1 18]]


In [61]:
pickle.dump(model, open("tree.sav", "wb"))

In [62]:
seed = 2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
train_test = (X_train, X_test, y_train, y_test)
k_fold = RepeatedKFold(n_splits=10, n_repeats=1, random_state=seed)
val_score = []
train_score = []
model = RandomForestClassifier(warm_start=True, n_estimators=100, max_features='auto')
for i, (train, val) in enumerate(k_fold.split(X_train)):
    print("Iteración:", i+1)
    print("val_size:", len(val))

    model.fit(X_train[train], y_train[train])
    model.n_estimators += 100

    score_val = model.score(X_train[val], y_train[val])
    val_score.append(score_val)
    score_train = model.score(X_train[train], y_train[train])
    train_score.append(score_train)
    print('Score val:', score_val)
    print('Score train:', score_train)
    print('##########################')

    if np.mean(val_score) > 0.99 and len(val_score) > 50:
        pickle.dump(model, open(path + "model_forest_warm_start", "wb"))
        print("STOP")
        break
    print('##########################')


Iteración: 1
val_size: 63
Score val: 0.8571428571428571
Score train: 1.0
##########################
##########################
Iteración: 2
val_size: 63
Score val: 1.0
Score train: 0.9982174688057041
##########################
##########################
Iteración: 3
val_size: 63
Score val: 1.0
Score train: 1.0
##########################
##########################
Iteración: 4
val_size: 63
Score val: 1.0
Score train: 1.0
##########################
##########################
Iteración: 5
val_size: 62
Score val: 1.0
Score train: 1.0
##########################
##########################
Iteración: 6
val_size: 62
Score val: 1.0
Score train: 1.0
##########################
##########################
Iteración: 7
val_size: 62
Score val: 1.0
Score train: 1.0
##########################
##########################
Iteración: 8
val_size: 62
Score val: 1.0
Score train: 1.0
##########################
##########################
Iteración: 9
val_size: 62
Score val: 1.0
Score train: 1.0
################

In [63]:
model.score(X_test, Y_test)

0.9102564102564102

RANDOM FOREST CONVERGES IN THE THRID ITERATION, WITH 300 STIMATORS. AFTER THAT, IT IS OVER TRAINING. THE RESULT IS EQUALY A 90% (SLIGHTLY UPPER)

## AUTOML
CITE: H2O.ai. (2022) h2o: R Interface for H2O. R package version 3.42.0.2. https://github.com/h2oai/h2o-3.

In [4]:
import h2o
from h2o.automl import H2OAutoML

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "15.0.2" 2021-01-19; Java(TM) SE Runtime Environment (build 15.0.2+7-27); Java HotSpot(TM) 64-Bit Server VM (build 15.0.2+7-27, mixed mode, sharing)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/r2/w54gxl1d7_130wxnhpdtlmhm0000gn/T/tmpcsvmi3zr
  JVM stdout: /var/folders/r2/w54gxl1d7_130wxnhpdtlmhm0000gn/T/tmpcsvmi3zr/h2o_juanbayonfernandez_started_from_python.out
  JVM stderr: /var/folders/r2/w54gxl1d7_130wxnhpdtlmhm0000gn/T/tmpcsvmi3zr/h2o_juanbayonfernandez_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_juanbayonfernandez_hexwmt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
train = data[data['player'] != 5]
test =  data[data['player'] == 5]
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
h2train['smash'] = h2train['smash'].asfactor()
h2test['smash'] = h2test['smash'].asfactor()

In [9]:
columnas = [a for a in h2train.columns if a != "smash"][1:]
x = columnas
y = "smash"

In [10]:
automl = H2OAutoML(max_models=50, seed=42, max_runtime_secs=300)
automl.train(x=x, y=y, training_frame=h2train, validation_frame=h2test)

AutoML progress: |
16:25:08.225: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
16:25:08.244: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_1_20231208_162508


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    60                 360                         116719                 4            7            6.98056       8             38            21.1806

ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 1.0213766916807404e-05
RMSE: 0.0031958984522051704
LogLoss: 0.001879267226341817
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2    3    4    5    Error    Rate
---  ---  ---  ---  ---  ---  -------  -------
150  0    0    0    0    0    0        0 / 150
0    150  0    0    0    0    0        0 / 150
0    0    75   0    0    0    0        0 / 75
0    0    0    75   0    0    0        0 / 75
0    0    0    0    100  0    0        0 / 100
0    0    0    0    0    100  0        0 / 100
150  150  75   75   100  100  0        0 / 650

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1
4    1
5    1
6    1

ModelMetricsMultinomial: gbm
** Reported on validation data. **

MSE: 0.08418653788473181
RMSE: 0.2901491648871866
LogLoss: 0.28552867492108525
Mean Per-Class Error: 0.1111111111111111
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2    3    4    5    Error      Rate
---  ---  ---  ---  ---  ---  ---------  --------
29   0    0    0    0    1    0.0333333  1 / 30
0    30   0    0    0    0    0          0 / 30
0    0    15   0    0    0    0          0 / 15
4    0    1    10   0    0    0.333333   5 / 15
0    0    0    0    20   0    0          0 / 20
1    5    0    0    0    14   0.3        6 / 20
34   35   16   10   20   15   0.0923077  12 / 130

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.907692
2    0.976923
3    0.992308
4    0.992308
5    1
6    1

ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.05302852304960816
RMSE: 0.23027922843714793
LogLoss: 0.1732096903659097
Mean Per-Class Error: 0.062222222222222213
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
0    1    2    3    4    5    Error      Rate
---  ---  ---  ---  ---  ---  ---------  --------
142  5    1    2    0    0    0.0533333  8 / 150
6    140  1    1    0    2    0.0666667  10 / 150
4    0    70   1    0    0    0.0666667  5 / 75
4    3    3    64   0    1    0.146667   11 / 75
0    0    0    0    97   3    0.03       3 / 100
1    0    0    0    0    99   0.01       1 / 100
157  148  75   68   97   105  0.0584615  38 / 650

Top-6 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.941538
2    0.98
3    0.995385
4    1
5    1
6    1

In [11]:
leader_board = automl.leaderboard
leader_board

model_id,mean_per_class_error,logloss,rmse,mse
GBM_2_AutoML_1_20231208_162508,0.0622222,0.17321,0.230279,0.0530285
GBM_3_AutoML_1_20231208_162508,0.0688889,0.19811,0.240943,0.0580533
GBM_4_AutoML_1_20231208_162508,0.0727778,0.186029,0.240083,0.0576397
GBM_5_AutoML_1_20231208_162508,0.0755556,0.202272,0.241134,0.0581455
GBM_grid_1_AutoML_1_20231208_162508_model_8,0.0761111,0.245501,0.253669,0.064348
GBM_grid_1_AutoML_1_20231208_162508_model_5,0.08,0.204018,0.251171,0.0630869
GBM_grid_1_AutoML_1_20231208_162508_model_4,0.0805556,0.222461,0.265315,0.070392
GBM_grid_1_AutoML_1_20231208_162508_model_2,0.0811111,0.232755,0.262878,0.0691047
GBM_grid_1_AutoML_1_20231208_162508_model_9,0.0883333,0.218789,0.260136,0.0676708
GBM_grid_1_AutoML_1_20231208_162508_model_10,0.09,0.217183,0.260383,0.0677991


In [18]:
pred = automl.predict(h2test)
pred


gbm prediction progress: |

███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1,p2,p3,p4,p5
0,0.451647,0.00849109,0.00274638,0.440453,0.00246294,0.0941992
0,0.841028,0.0622694,0.00184136,0.0325844,0.00292564,0.0593518
0,0.990204,0.000106265,9.791e-05,0.000416227,5.48927e-05,0.00912097
0,0.998115,0.000103447,0.000256209,0.000274215,5.15906e-05,0.00119966
0,0.985868,0.0013017,0.00117451,0.000763049,0.000448051,0.0104452
0,0.997186,0.000136379,8.89926e-05,0.000162786,6.5693e-05,0.00236052
0,0.995525,0.000207633,0.000169368,0.000298189,0.000122168,0.00367791
0,0.996699,0.000130332,0.000226806,0.000256428,8.4217e-05,0.0026031
0,0.994722,0.000231982,0.000636167,0.00045164,0.000137058,0.00382076
0,0.481833,0.0137793,0.0184137,0.306925,0.00760053,0.171449


In [22]:
best = automl.get_best_model()
model_path = h2o.save_model(model=best, path="./", force=True)

## CONCLUSION

GBM AS BETTER MODEL. THE COLUMNS GENERATED DISTANCE, ANGLE AND POINTS OF THE TRAJECTORY DO NOT MATTER FOR THE OUTOCOME, AND CAN BE DELETED. 